In [ ]:
from dataloader import *
from model import *
from plot_utils import *
from torch.cuda.amp import autocast as autocast
import matplotlib.pyplot as plt
from medpy.metric import binary
import numpy as np


domain_list=['./FAZ/Domain1','./FAZ/Domain2','./FAZ/Domain3','./FAZ/Domain4','./FAZ/Domain5']

domain_list=domain_list
input_folderL=[os.path.join(domain_list[i],'test/imgs') for i in range(len(domain_list))]
label_folderL=[os.path.join(domain_list[i],'test/mask') for i in range(len(domain_list))]
pred_folderL=[os.path.join(domain_list[i],'test/pred') for i in range(len(domain_list))]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

save_path = './checkpoint/domain1_w_val4.pth'
model_state = torch.load(save_path)


model = UNet(in_channels=3, out_channels=1)
model.load_state_dict(model_state)
model.eval()
model.to(device,dtype=torch.float32)

In [ ]:
with torch.no_grad():
    
    # one domain at a time
    for idx in range(len(domain_list)):
        diceScore=0
        dic_record=[]
        hd95_record=[]

        test_data_loader = get_Domain1_Dataloader(input_folder=input_folderL[idx], label_folder=label_folderL[idx], batch_size=6)

        
        print(f"domain:{domain_list[idx]}\n")
        for index, (inputs, labels) in enumerate(test_data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.eval()
            outputs = model(inputs)
            diceScore = dice_coefficient(outputs, labels)
            dic_record.append(diceScore)
            show_result(inputs, outputs, labels)
            # plot_some(inputs.cpu(),labels.cpu(),outputs.cpu())

        torch.cuda.empty_cache()